### Few shot prompting
Use sys prompt 1.
N-1 hold out.
Models: GPT-3.5 (16k), GPT-4 (16k), CodeLlama-Instruct-7B (16k), CodeLlama-Instruct-34B (16k).

In [1]:
import data
import evaluation

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/omkar/.config/sagemaker/config.yaml


In [3]:
dataset = data.Data("config_1.json", "nagini_examples/list_master_finetune")
ev = evaluation.Evaluation(dataset, "gpt-4")
ev.run_eval(k=3, n=3, key="list")

Running example: join ; attempt: 1 ; error depth: 1
Using temperature: 0.0
Generated program from GPT:
def join(a: Optional[Node], b: Optional[Node], c: Optional[Node]) -> None:
    """Join two list segments."""
    Requires(lseg(a, b) and lseg(b, c))
    Ensures(lseg(a, c))
    if b is c:
        return
    if a is b:
        return
    Unfold(lseg(a, b))
    join(a.next, b, c)
    Fold(lseg(a, c))
response ['', 'Verification successful', 'Verification took 1.82 seconds.']
Verification result:
 Verification successful 


Running example: prepend ; attempt: 1 ; error depth: 1
Using temperature: 0.0
Generated program from GPT:
def prepend(head: Optional[Node], val: int) -> Node:
    """Prepend a new node with value val to the list."""
    Requires(lseg(head, None))
    Ensures(lseg(Result(), None))
    n = Node(val, head)
    Fold(lseg(n, None))
    return n
response ['', 'Verification successful', 'Verification took 1.88 seconds.']
Verification result:
 Verification successful 


Runni

EvalResult(results={'join': True, 'prepend': True, 'remove_first': True, 'remove_last': False, 'contains': True, 'contains_iter': True, 'insert': True, 'insert_iter': True, 'append': True, 'append_iter': False, 'index_of': True, 'reverse': False, 'insert_sorted': True, 'insertion_sort': False, 'insertion_sort_iter': False, 'merge': True, 'count': True, 'count_iter': True, 'split': True, 'split_iter': True, 'merge_sort': True}, verified_at={'join': (1, 1), 'prepend': (1, 1), 'remove_first': (3, 2), 'contains': (2, 1), 'contains_iter': (1, 1), 'insert': (1, 1), 'insert_iter': (3, 1), 'append': (1, 1), 'index_of': (1, 1), 'insert_sorted': (2, 1), 'merge': (1, 1), 'count': (1, 1), 'count_iter': (1, 1), 'split': (1, 1), 'split_iter': (1, 1), 'merge_sort': (1, 2)})

### Incremental verification
Two experiments with two different system prompts.
1. The detailed system prompt
2. The basic system prompt

In [9]:
dataset = data.Data("config.json", "final/list_incremental_gpt4turbo")
import random
rand = random.Random(42)
examples = dataset.get_list_of_examples("list")
# rand.shuffle(examples)
# rand.shuffle(examples)
examples

['prepend',
 'join_lists',
 'contains',
 'merge',
 'append',
 'insert_sorted',
 'remove',
 'remove_first',
 'remove_last',
 'index_of',
 'insert',
 'drop',
 'drop_iter',
 'reverse',
 'insertion_sort',
 'count',
 'split',
 'merge_sort']

In [10]:
dataset = data.Data("config.json", "final/list_incremental_gpt4turbo")
ev = evaluation.Evaluation(dataset, "gpt-4")

KEY = "list"
unverified = dataset.get_list_of_examples(KEY)
verified = []
unverified = [x for x in unverified if x not in verified]
rand.shuffle(unverified)
rand.shuffle(unverified)
print(unverified)

verified_updated = True
while len(unverified) > 0 and verified_updated:
    for example in unverified[:]:
        if not all([x in verified for x in dataset.get_dependencies(KEY, example)]):
            print(f"Skipping {example} as not all dependencies verified")
            continue
        if len(verified) > 4:
            k, n = 3, 3
        else:
            k, n = 3,3
        verifies, _, attempt = ev.run_example(example, k=k, n=n, key=KEY, examples=verified)
        if verifies:
            unverified.remove(example)
            verified.append(example)
            dataset.add_verified(example, attempt, key=KEY)
            break
        else:
            # push it to the end of unverified
            unverified.remove(example)
            unverified.append(example)
    if not verifies: verified_updated = False

['drop', 'merge_sort', 'join_lists', 'insert', 'remove_last', 'drop_iter', 'contains', 'remove_first', 'prepend', 'append', 'reverse', 'insert_sorted', 'count', 'remove', 'index_of', 'insertion_sort', 'split', 'merge']
Running example: drop ; attempt: 1 ; error depth: 1
Using temperature: 0.1
Generated program from GPT:
def drop(head: Optional[Node], val: int) -> Optional[Node]:
    """Drops nodes until val is found and returns the list starting at val node, or None if not found.
    Permissions to the list until val node are leaked."""
    Requires(is_list(head))
    Ensures(Implies(Result() is not None, is_list(Result())))
    if head is None:
        return None
    Unfold(is_list(head))
    if head.val == val:
        Fold(is_list(head))
        return head
    result = drop(head.next, val)
    Fold(is_list(head))
    return result
response ['', 'Verification failed', 'Errors:', 'Fold might fail. There might be insufficient permission to access is_list(head.next). (tmp.py@33.4)', '

In [8]:
print(verified)

['drop_iter', 'append', 'index_of', 'insert_sorted', 'remove_first', 'split', 'remove', 'count', 'join_lists', 'drop', 'contains', 'insertion_sort', 'remove_last', 'prepend', 'insert', 'merge', 'merge_sort', 'reverse']


In [ ]:
dataset = data.Data("config_eval.json", "nagini_examples/lseg_master_finetune")
ev = evaluation.Evaluation(dataset, "CodeLlama-34b-Instruct-hf")
ev.run_eval(k=3, n=3, key="lseg")
# running with full sys prompt -- already ran.

In [ ]:
dataset = data.Data("config_eval_1.json", "nagini_examples/lseg_master_finetune")
ev = evaluation.Evaluation(dataset, "CodeLlama-34b-Instruct-hf")
ev.run_eval(k=3, n=3, key="lseg")
# running with basic sys prompt

In [19]:
import jsonlines
# print contents of evalresults_pretrained.txt
# each entry is of the form {'list_prepend': '    Ensures(is_list(Result()))\n    n = Node(val, head)\n    return n\n\n\n### Verification error:\nVerification failed: Postcondition of prepend might not hold. There might be insufficient permission to access is_list(Result()). at line 3.12\n\n### Verified program:\ndef prepend(head: Node, val: int) -> Node:\n    """Prepends a new node with the given value to the list."""\n    Ensures(is_list(Result()))\n    n = Node(val, head)\n    return n\n\n\n### Verification error:\nVerification failed: Postcondition of prepend might not hold. There might be insufficient permission to access is_list(Result()). at line 3.12\n\n### Verified program:\ndef prepend(head: Node, val: int) -> Node:\n    """Prepends a new node with the given value to the list."""\n    Ensures(is_list(Result()))\n    n = Node(val, head)\n    return n\n\n\n### Verification error'}
# print only the value
with jsonlines.open("evalresults_pretrained.txt") as reader:
    for obj in reader:
        print(obj[list(obj.keys())[0]])
        print("======")

    Ensures(is_list(Result()))
    n = Node(val, head)
    return n


### Verification error:
Verification failed: Postcondition of prepend might not hold. There might be insufficient permission to access is_list(Result()). at line 3.12

### Verified program:
def prepend(head: Node, val: int) -> Node:
    """Prepends a new node with the given value to the list."""
    Ensures(is_list(Result()))
    n = Node(val, head)
    return n


### Verification error:
Verification failed: Postcondition of prepend might not hold. There might be insufficient permission to access is_list(Result()). at line 3.12

### Verified program:
def prepend(head: Node, val: int) -> Node:
    """Prepends a new node with the given value to the list."""
    Ensures(is_list(Result()))
    n = Node(val, head)
    return n


### Verification error
    if head.next is None:
        n = Node(val)
        head.next = n
    else:
        append(head.next, val)


### Verification error:
Verification failed: Conditional sta